In [1]:
import pickle

import tarfile
from tqdm import tqdm_notebook as tqdm

from multiprocessing import Pool


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('russian')

from string import punctuation
# punkt= [p for p in punctuation] + ["`", "``" ,"''", "'"]
punkt= [p for p in punctuation] + ["`", "``" ,"''", "'", '"','""','"""','«','»']


import gensim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
import pymorphy3

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

In [ ]:

# # Загрузка данных из файла
# df = pd.read_csv('shorted_data_combined.csv')



# # Удаление пунктуации и стоп-слов из колонки 'topic'
# df['topic'] = df['topic'].apply(
#     lambda x: ' '.join([word.lower() for word in nltk.word_tokenize(x) if word.lower() not in stop and word not in punkt])
# )

# # Сохранение измененных данных в новый файл
# df.to_csv('del_pun_stop_words.csv', index=False)

In [3]:
df = pd.read_csv('del_pun_stop_words.csv')

In [4]:
df.count()

title     9802
topic    10695
dtype: int64

In [8]:
df2 = df.dropna()



In [ ]:
df2.info()

In [13]:
# Задаем максимальное количество новостей для каждой темы
max_news_per_topic = 1000

# Группируем данные по теме и выбираем не более 1000 новостей из каждой темы
df2 = df2.groupby('topic').sample(max_news_per_topic)

In [16]:
df.head(3)

,title,topic
0,Министр регионального развития Грузии ушел в о...,бывший ссср
1,В Киеве допустили ракетный удар России по Украине,бывший ссср
2,Заместитель Тимошенко пригрозил фальсификатора...,бывший ссср


In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9000 entries, 70 to 6652
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9000 non-null   object
 1   topic   9000 non-null   object
dtypes: object(2)
memory usage: 210.9+ KB


In [17]:
# topic_counts = df['topic'].value_counts(normalize=True)
# print(topic_counts)

topic_counts = df2['topic'].value_counts()
topic_percentage = df2['topic'].value_counts(normalize=True) * 100

result_df = pd.DataFrame({
    'Topic': topic_counts.index,
    'Count': topic_counts.values,
    'Percentage': topic_percentage.values
})

print(result_df)



               Topic  Count  Percentage
0        бывший ссср   1000   11.111111
1             забота   1000   11.111111
2      наука техника   1000   11.111111
3        путешествия   1000   11.111111
4             россия   1000   11.111111
5  силовые структуры   1000   11.111111
6              спорт   1000   11.111111
7      строительство   1000   11.111111
8          экономика   1000   11.111111


    'Россия': '0',
    'Экономика': '1',
    'Силовые структуры': '2',
    'Бывший СССР': '3',
    'Спорт': '4',
    'Забота': '5',
    'Строительство': '6',
    'Путешествия': '7',
    'Наука техника': '8'

### Пример:

In [ ]:
from pymorphy3 import MorphAnalyzer
import pymorphy3_dicts_ru
# Создание объекта лемматизатора
lemmatizer = MorphAnalyzer(path=pymorphy3_dicts_ru.get_path(),lang='ru')

# Пример текста для лемматизации
text1 = "Кошки любят  лазить по деревьям."
text2 = "Кошки любят  лазить по деревьям."
text = ' br '.join([text1,text2])
# Лемматизация слов в тексте
lemmatized_text = ' '.join([lemmatizer.parse(word)[0].normal_form for word in text.split()])
print(lemmatized_text.split('br'))

In [40]:
from pymorphy3 import MorphAnalyzer
import pymorphy3_dicts_ru
# import pandas as pd

# Создание объекта лемматизатора
lemmatizer = MorphAnalyzer(path=pymorphy3_dicts_ru.get_path(), lang='ru')

# Разбивка DataFrame на блоки по 500 строк
block_size = 500

num_blocks = len(df2) // block_size 
cheker = 0
# # Новый DataFrame для хранения лемматизированных данных
lemmatized_df = pd.DataFrame(columns=['title', 'topic'])
checker = 0
for i in range(num_blocks):
    start_idx = i * block_size
    end_idx = (i + 1) * block_size


    block_data = df2.iloc[start_idx:end_idx]

    # Объединение текста в блоке через 'br'
    block_text = ' ||| '.join(block_data['title'])

    # # Лемматизация слов в тексте
    lemmatized_text = ' '.join([lemmatizer.parse(word)[0].normal_form for word in block_text.split(' ')])
    lemmatized_text = lemmatized_text.split('|||')
    # проверяем на всякий
    # print(len(lemmatized_text),lemmatized_text)
  


#     # Создание DataFrame для блока с лемматизированными данными
    lemmatized_block_df = pd.DataFrame({'title': lemmatized_text, 'topic': block_data['topic']})
    # checker += 
    # print(lemmatized_block_df.count())
#     # Добавление лемматизированных данных в основной DataFrame
    lemmatized_df = pd.concat([lemmatized_df, lemmatized_block_df])
    print(lemmatized_df.count())

# # # Вывод первых 3 строк нового DataFrame
# print(lemmatized_df.head(3))


print(cheker)

title    500
topic    500
dtype: int64
title    1000
topic    1000
dtype: int64
title    1500
topic    1500
dtype: int64
title    2000
topic    2000
dtype: int64
title    2500
topic    2500
dtype: int64
title    3000
topic    3000
dtype: int64
title    3500
topic    3500
dtype: int64
title    4000
topic    4000
dtype: int64
title    4500
topic    4500
dtype: int64
title    5000
topic    5000
dtype: int64
title    5500
topic    5500
dtype: int64
title    6000
topic    6000
dtype: int64
title    6500
topic    6500
dtype: int64
title    7000
topic    7000
dtype: int64
title    7500
topic    7500
dtype: int64
title    8000
topic    8000
dtype: int64
title    8500
topic    8500
dtype: int64
title    9000
topic    9000
dtype: int64
0


In [41]:
lemmatized_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9000 entries, 70 to 6652
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9000 non-null   object
 1   topic   9000 non-null   object
dtypes: object(2)
memory usage: 210.9+ KB


In [61]:
topic_mapping = {
    'Россия': '0',
    'Экономика': '1',
    'Силовые структуры': '2',
    'Бывший СССР': '3',
    'Спорт': '4',
    'Забота': '5',
    'Строительство': '6',
    'Путешествия': '7',
    'Наука техника': '8'
}
topic_mapping = {key.lower(): value for key, value in topic_mapping.items()}

# Заменить значения в колонке 'topic' на их числовые представления из словаря
lemmatized_df['topic'] = lemmatized_df['topic'].replace(topic_mapping)

In [62]:
lemmatized_df.head()

,title,topic
70,мэр ивано-франковск присоединиться к бессрочны...,3
758,южный осетия выставлять грузия счёт на 18 мил...,3
0,министр региональный развитие грузия уйти в о...,3
535,чичваркин прилететь в киев для проведение мас...,3
774,донецкий фильтровальный станция обесточить из...,3


In [63]:
titles = lemmatized_df['title']
topics = lemmatized_df['topic']
X = titles.tolist()
Y = topics.tolist()

In [64]:
title_train, title_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=33)

In [65]:
with open('train_data_titles.txt', 'w+', encoding='utf-8') as tr:
    for idx in range(len(title_train)):
        # tr.write('__label__' + y_train[idx]+' '+title_train[idx]+'\n')
        tr.write('__label__' + f'{y_train[idx]}'+' '+title_train[idx]+'\n')
        
with open('test_data_titles.txt', 'w+', encoding='utf-8') as te:
    for idx in range(len(title_test)):
        # te.write('__label__' + y_test[idx]+' '+title_test[idx]+'\n')        
        te.write('__label__' + f'{y_test[idx]}'+' '+title_test[idx]+'\n')    
print('Все записано!')

Все записано!


In [66]:
%%time
ft_model = fasttext.train_supervised(input='train_data_titles.txt')

CPU times: total: 531 ms
Wall time: 259 ms


In [68]:
def tokenize(sent):
    try:
        sent = word_tokenize(sent)
        return [word for word in sent if word not in stop and word not in punkt]
    
    except:
        return []
    
def lemmatize(sent):
    try:
        return " ".join([lemmatizer.normal_forms(word)[0] for word in sent])
    except:
        return " "

    
# def preprocess_sent(sent):
#     return lemmatize(tokenize(sent))


def preprocess_sent(sent):
    try:
        tokenized_sent = tokenize(sent)
        lemmatized_sent = lemmatize(tokenized_sent)
        return lemmatized_sent
    except Exception as e:
        print(f"Error in preprocess_sent: {e}")
        return " "

In [70]:
ft_model.predict(preprocess_sent('Акула съела банкира: акции банка упали на 25%'), k=3)

(('__label__1', '__label__8', '__label__2'),
 array([0.36269757, 0.13101771, 0.13025342]))

In [71]:
%%time
ft_model_p1 = fasttext.train_supervised(input='train_data_titles.txt', epoch=20, wordNgrams=2, loss='hs', lr=1.0)

CPU times: total: 8.41 s
Wall time: 1.92 s


In [72]:
ft_model_p1.test('test_data_titles.txt')

(1800, 0.6955555555555556, 0.6955555555555556)

In [91]:
%%time
# ft_model_p2 = fasttext.train_supervised(input='train_data_titles.txt', autotuneValidationFile='test_data_titles.txt')

CPU times: total: 22min 11s
Wall time: 6min 10s


In [92]:
# ft_model_p2.test('test_data_titles.txt')

(1800, 0.76, 0.76)

In [93]:
# print(f'Best learning rate: {ft_model_p2.lr} \nBest epochs: {ft_model_p2.epoch} \nBest_word_Ngrams: {ft_model_p2.wordNgrams}')

Best learning rate: 0.3761170672491951 
Best epochs: 48 
Best_word_Ngrams: 1


### Проверка:

In [73]:
# test_data = pd.read_csv('test_news.csv')

In [77]:
# test_data.head()

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...


In [74]:
# test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26275 entries, 0 to 26274
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  26275 non-null  object
dtypes: object(1)
memory usage: 205.4+ KB


In [78]:

# # Удаление пунктуации и стоп-слов из колонки 'topic'
# test_data['content'] = test_data['content'].apply(
#     lambda x: ' '.join([word.lower() for word in nltk.word_tokenize(x) if word.lower() not in stop and word not in punkt])
# )

# # Сохранение измененных данных в новый файл
# # test_data.to_csv('del_psw_test_news.csv', index=False)

In [81]:
# test_data['content'].shape[0]

26275

In [83]:
# from pymorphy3 import MorphAnalyzer
# import pymorphy3_dicts_ru
# # import pandas as pd

# # Создание объекта лемматизатора
# lemmatizer = MorphAnalyzer(path=pymorphy3_dicts_ru.get_path(), lang='ru')

# # Разбивка DataFrame на блоки по 500 строк
# block_size = 1000

# num_blocks = len(test_data) // block_size 

# # # Новый DataFrame для хранения лемматизированных данных
# lemmatized_df = pd.DataFrame(columns=['content'])

# for i in range(num_blocks):
#     start_idx = i * block_size
#     end_idx = (i + 1) * block_size
#     if end_idx == 26000:
#         end_idx = test_data['content'].shape[0]
#     # print(i,start_idx,end_idx)

#     block_data = test_data.iloc[start_idx:end_idx]

#     # Объединение текста в блоке через 'br'
#     block_text = ' ||| '.join(block_data['content'])

#     # # Лемматизация слов в тексте
#     lemmatized_text = ' '.join([lemmatizer.parse(word)[0].normal_form for word in block_text.split(' ')])
#     lemmatized_text = lemmatized_text.split('|||')
#     # проверяем на всякий
#     # print(len(lemmatized_text),lemmatized_text)
  


# #     # Создание DataFrame для блока с лемматизированными данными
#     lemmatized_block_df = pd.DataFrame({'content': lemmatized_text})
#     # print(lemmatized_block_df.count())
# #     # Добавление лемматизированных данных в основной DataFrame
#     lemmatized_df = pd.concat([lemmatized_df, lemmatized_block_df])
#     print(lemmatized_df.count())



content    1000
dtype: int64
content    2000
dtype: int64
content    3000
dtype: int64
content    4000
dtype: int64
content    5000
dtype: int64
content    6000
dtype: int64
content    7000
dtype: int64
content    8000
dtype: int64
content    9000
dtype: int64
content    10000
dtype: int64
content    11000
dtype: int64
content    12000
dtype: int64
content    13000
dtype: int64
content    14000
dtype: int64
content    15000
dtype: int64
content    16000
dtype: int64
content    17000
dtype: int64
content    18000
dtype: int64
content    19000
dtype: int64
content    20000
dtype: int64
content    21000
dtype: int64
content    22000
dtype: int64
content    23000
dtype: int64
content    24000
dtype: int64
content    25000
dtype: int64
content    26275
dtype: int64


In [84]:
# lemmatized_df.to_csv('lemm_test_news.csv', index=False)

In [95]:
test_data = pd.read_csv('lemm_test_news.csv')

In [94]:
# Предсказание меток для каждого заголовка новости
predicted_labels = []


In [96]:
for title in test_data['content']:  # Замените 'content' на реальное имя столбца с заголовками
    # Удаление символа новой строки, если он присутствует
    title = title.replace('\n', ' ')
    # Предсказание меток
    # labels, probs = ft_model_p1.predict(title)
    labels, probs = ft_model_p2.predict(title)
    # print(labels, probs)
    predicted_labels.append(labels[0])

# Добавление предсказанных меток в тестовые данные
test_data['predicted_label'] = predicted_labels

# Вывод результата
# print(test_data[['content', 'predicted_label']])

In [97]:
test_data.head()

,content,predicted_label
0,фото фонтанка.р поделитьсяэкс-министр оборона ...,__label__2
1,начало февраль 2023 год пушкинский район санк...,__label__6
2,фото andy bao getty images анастасия борисов ...,__label__4
3,хотеть съездить море летом-2023 — читать далё...,__label__5
4,сергей пиняев фото алексей филипп риа новость...,__label__4


In [98]:
# Создание DataFrame с результатами
pre_result_df = pd.DataFrame({'content': test_data['content'], 'predicted_label': test_data['predicted_label']})

In [99]:

pre_result_df['topic'] = test_data['predicted_label'].copy().str.replace('__label__', '').astype(int)
pre_result_df['index'] = test_data.index.copy()
pre_result_df.head()

,content,predicted_label,topic,index
0,фото фонтанка.р поделитьсяэкс-министр оборона ...,__label__2,2,0
1,начало февраль 2023 год пушкинский район санк...,__label__6,6,1
2,фото andy bao getty images анастасия борисов ...,__label__4,4,2
3,хотеть съездить море летом-2023 — читать далё...,__label__5,5,3
4,сергей пиняев фото алексей филипп риа новость...,__label__4,4,4


In [100]:
result_df = pre_result_df[['topic','index']].copy()

result_df.head()

,topic,index
0,2,0
1,6,1
2,4,2
3,5,3
4,4,4


In [101]:
result_df.to_csv('result_3.csv', index=False)